In [1]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
connection = create_connection("SQL_booking_s_4.db")

Connection to SQLite DB successful


In [4]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

## Подготовка дашборда

In [5]:
!pip install dash

In [6]:
!pip install jupyter-dash

In [7]:
!pip install pandas

### Просмотр названий таблиц из базы данных

In [8]:
table = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

         name
0      Orders
1      Review
2       Users
3  Apart_info


### Просмотр полей таблиц из базы данных

In [9]:
c = connection.cursor()

def table_info(c, conn):
    '''
    prints out all of the columns of every table in db
    c : cursor object
    conn : database connection object
    '''
    tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0] # tables is a list of single item tuples
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
        print()

table_info(c, connection)

Orders
	Booking_ID
	no_of_adults
	no_of_children
	no_of_weekend_nights
	no_of_week_nights
	type_of_meal_plan
	required_car_parking_space
	room_type_reserved
	lead_time
	year
	month
	day
	market_segment_type
	repeated_guest
	no_of_previous_cancellations
	no_of_previous_bookings_not_canceled
	avg_price_per_room
	no_of_special_requests
	booking_status
	user_id
	date_arrival
	apart_id

Review
	index
	Uniq Id
	Source
	Hotel Id From Source
	Hotel Url From Source
	Hotel Name
	Review Url
	Review Publication Date
	Review Author Type
	Review Author Name
	Review Author Location
	Review Author Age
	Review Author Count
	Review Stay Details
	Review Title
	Review Text Positive
	Review Text Negative
	Review Rating

Users
	ID
	Nationality
	Age
	DaysSinceCreation
	NameHash
	DocIDHash
	AverageLeadTime
	LodgingRevenue
	OtherRevenue
	BookingsCanceled
	BookingsNoShowed
	BookingsCheckedIn
	PersonsNights
	RoomNights
	DaysSinceLastStay
	DaysSinceFirstStay
	DistributionChannel
	MarketSegment
	SRHighFloor
	SRLow

In [10]:
meta = c.execute("PRAGMA table_info('Review')")
for r in meta:
    print(r)

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'Uniq Id', 'TEXT', 0, None, 0)
(2, 'Source', 'TEXT', 0, None, 0)
(3, 'Hotel Id From Source', 'REAL', 0, None, 0)
(4, 'Hotel Url From Source', 'TEXT', 0, None, 0)
(5, 'Hotel Name', 'TEXT', 0, None, 0)
(6, 'Review Url', 'TEXT', 0, None, 0)
(7, 'Review Publication Date', 'TEXT', 0, None, 0)
(8, 'Review Author Type', 'TEXT', 0, None, 0)
(9, 'Review Author Name', 'TEXT', 0, None, 0)
(10, 'Review Author Location', 'TEXT', 0, None, 0)
(11, 'Review Author Age', 'TEXT', 0, None, 0)
(12, 'Review Author Count', 'REAL', 0, None, 0)
(13, 'Review Stay Details', 'TEXT', 0, None, 0)
(14, 'Review Title', 'TEXT', 0, None, 0)
(15, 'Review Text Positive', 'TEXT', 0, None, 0)
(16, 'Review Text Negative', 'TEXT', 0, None, 0)
(17, 'Review Rating', 'REAL', 0, None, 0)


In [11]:
data = pd.read_sql('SELECT count() FROM orders', connection)
data

count()
0    36275

In [12]:
for i in ['orders', 'Review', 'Users', 'Apart_info']:
    data = pd.read_sql(f'SELECT count() FROM {i}', connection)
    print(i)
    display(data)

orders


count()
0    36275

Review


count()
0    30000

Users


count()
0    83590

Apart_info


count()
0     5891

In [13]:
col_name_o = list(pd.read_sql('SELECT * FROM orders', connection).columns)
col_name_o

['Booking_ID',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'type_of_meal_plan',
 'required_car_parking_space',
 'room_type_reserved',
 'lead_time',
 'year',
 'month',
 'day',
 'market_segment_type',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests',
 'booking_status',
 'user_id',
 'date_arrival',
 'apart_id']

In [14]:
col_sql_o = 'orders.' + col_name_o[0] + ', ' + 'orders.' + ', orders.'.join(col_name_o[1:])
col_sql_o

'orders.Booking_ID, orders.no_of_adults, orders.no_of_children, orders.no_of_weekend_nights, orders.no_of_week_nights, orders.type_of_meal_plan, orders.required_car_parking_space, orders.room_type_reserved, orders.lead_time, orders.year, orders.month, orders.day, orders.market_segment_type, orders.repeated_guest, orders.no_of_previous_cancellations, orders.no_of_previous_bookings_not_canceled, orders.avg_price_per_room, orders.no_of_special_requests, orders.booking_status, orders.user_id, orders.date_arrival, orders.apart_id'

In [15]:
col_name_u = list(pd.read_sql('SELECT * FROM Users', connection).columns)
col_name_u

['ID',
 'Nationality',
 'Age',
 'DaysSinceCreation',
 'NameHash',
 'DocIDHash',
 'AverageLeadTime',
 'LodgingRevenue',
 'OtherRevenue',
 'BookingsCanceled',
 'BookingsNoShowed',
 'BookingsCheckedIn',
 'PersonsNights',
 'RoomNights',
 'DaysSinceLastStay',
 'DaysSinceFirstStay',
 'DistributionChannel',
 'MarketSegment',
 'SRHighFloor',
 'SRLowFloor',
 'SRAccessibleRoom',
 'SRMediumFloor',
 'SRBathtub',
 'SRShower',
 'SRCrib',
 'SRKingSizeBed',
 'SRTwinBed',
 'SRNearElevator',
 'SRAwayFromElevator',
 'SRNoAlcoholInMiniBar',
 'SRQuietRoom']

In [16]:
col_sql_u = 'Users.' + col_name_u[0] + ', ' + 'Users.' + ', Users.'.join(col_name_u[1:])
col_sql_u

'Users.ID, Users.Nationality, Users.Age, Users.DaysSinceCreation, Users.NameHash, Users.DocIDHash, Users.AverageLeadTime, Users.LodgingRevenue, Users.OtherRevenue, Users.BookingsCanceled, Users.BookingsNoShowed, Users.BookingsCheckedIn, Users.PersonsNights, Users.RoomNights, Users.DaysSinceLastStay, Users.DaysSinceFirstStay, Users.DistributionChannel, Users.MarketSegment, Users.SRHighFloor, Users.SRLowFloor, Users.SRAccessibleRoom, Users.SRMediumFloor, Users.SRBathtub, Users.SRShower, Users.SRCrib, Users.SRKingSizeBed, Users.SRTwinBed, Users.SRNearElevator, Users.SRAwayFromElevator, Users.SRNoAlcoholInMiniBar, Users.SRQuietRoom'

In [20]:
data = pd.read_sql(f'SELECT o.Booking_ID, o.no_of_adults, DATE(o.date_arrival) as date, U.ID, U.Nationality, U.Age FROM orders as o LEFT JOIN Users as U ON o.user_id = U.ID', connection)
data

Booking_ID  no_of_adults        date       ID Nationality   Age
0       INN00001             2  2017-10-02   4696.0         CRI  33.0
1       INN00002             2  2018-11-06  65881.0         GBR  40.0
2       INN00003             1  2018-02-28  76735.0         DEU  53.0
3       INN00004             2  2018-05-20  33602.0         CAN  37.0
4       INN00005             2  2018-04-11   8939.0         SRB  30.0
...          ...           ...         ...      ...         ...   ...
36270   INN36271             3  2018-08-03  76258.0         ESP  40.0
36271   INN36272             2  2018-10-17  63267.0         USA  17.0
36272   INN36273             2  2018-07-01  28930.0         PRT  67.0
36273   INN36274             2  2018-04-21  43293.0         ESP  38.0
36274   INN36275             2  2018-12-30  36142.0         CAN  58.0

[36275 rows x 6 columns]

In [115]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()

df1 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2017 GROUP BY date(date_arrival)', connection)

df2 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2018 GROUP BY date(date_arrival)', connection)

fig = px.box(df1, y="booking_number")

app.layout = html.Div(children=[
    html.H1(children='Бронирование жилья', style={'text-align': 'center'}),
    html.Div(children='Динамика бронирований по датам', style={'text-align': 'center'}),
    
    html.Div([
        html.Label(['Выберите год:'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': '2017 год', 'value': 'graph1'},
                {'label': '2018 год', 'value': 'graph2'},
                    ],
            value='graph1',
            style={"width": "60%"}),
        
    html.Div(dcc.Graph(id='graph')),        
        ]),
    
    html.Div(dcc.Graph(id='boxplot')),        
        ])
    



@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == 'graph1':
        fig1 = px.bar(df1, x=df1['date'], y=df1['booking_number'])
        return fig1
    else:
        fig2 = px.bar(x=df2['date'], y=df2['booking_number'])
        return fig2
    


app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2023 20:45:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_favicon.ico?v=2.9.1 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:47:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:47:29] "POST /_dash-update-component HTTP/1.1" 

In [115]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()

df1 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2017 GROUP BY date(date_arrival)', connection)

df2 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2018 GROUP BY date(date_arrival)', connection)

fig = px.box(df1, y="booking_number")

app.layout = html.Div(children=[
    html.H1(children='Бронирование жилья', style={'text-align': 'center'}),
    html.Div(children='Динамика бронирований по датам', style={'text-align': 'center'}),
    
    html.Div([
        html.Label(['Выберите год:'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': '2017 год', 'value': 'graph1'},
                {'label': '2018 год', 'value': 'graph2'},
                    ],
            value='graph1',
            style={"width": "60%"}),
        
    html.Div(dcc.Graph(id='graph')),        
        ]),
    
    html.Div(dcc.Graph(id='boxplot')),        
        ])
    



@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == 'graph1':
        fig1 = px.bar(df1, x=df1['date'], y=df1['booking_number'])
        return fig1
    else:
        fig2 = px.bar(x=df2['date'], y=df2['booking_number'])
        return fig2
    
@app.callback(
    Output('boxplot', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == 'graph1':
        fig1 = px.box(df1, y='booking_number', title = '"Ящик с усами" для числа бронирований')
        return fig1
    else:
        fig2 = px.box(df2, y='booking_number', title = '"Ящик с усами" для числа бронирований')
        return fig2
    


app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2023 20:45:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_favicon.ico?v=2.9.1 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:45:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:47:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 20:47:29] "POST /_dash-update-component HTTP/1.1" 

In [ ]:
connection.close()